In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
data = pd.read_feather('target_stocks.feather')
data.tail(20)

,index,stock_list
139,2024-05-22,[3019]
140,2024-05-23,[00662]
141,2024-05-30,"[2451, 5536, 6285, 2524]"
142,2024-06-03,[3596]
143,2024-06-11,[3532]
144,2024-06-14,[00662]
145,2024-06-21,[6023]
146,2024-06-24,"[2801, 5234]"
147,2024-07-02,"[3189, 6213, 2539]"
148,2024-07-04,"[3023, 2897, 2801]"


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import logging
import yaml
from daily_bt import (
    load_config, 
    read_stock_data, 
    construct_data, 
    execute_short_strategy,
    calculate_daily_performance,
    backtest_single_stock,
    backtest_multiple_stocks, 
    get_stocks_for_date, 
    main
)


In [4]:
config = load_config('config.yaml')

In [5]:
df= read_stock_data('5536', datetime.strptime('2023-11-16', '%Y-%m-%d'), '/Users/mouyasushi/k_data/永豐')
df.head()

,Open,High,Low,Close,Volume,Amount
ts,,,,,,
2023-11-16 09:21:00,176.0,176.0,176.0,176.0,1,176000.0
2023-11-16 09:25:00,176.0,176.0,176.0,176.0,1,176000.0
2023-11-16 09:28:00,176.0,176.0,176.0,176.0,2,352000.0
2023-11-16 09:32:00,176.0,176.0,176.0,176.0,1,176000.0
2023-11-16 09:33:00,176.0,176.0,176.0,176.0,1,176000.0


In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
np_data= construct_data(df, ma_period=14) # this is a dictioinary 
open_np = np_data['Close']
print(open_np)
print(len(open_np))


[  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.  176.    0.    0.    0.
 176.    0.    0.  176.    0.    0.    0.  176.  176.    0.    0.    0.
   0.    0.  175.5   0.  175.5   0.    0.    0.  175.5 175.5   0.    0.
   0.  176.    0.    0.    0.    0.    0.    0.    0.  176.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  175.5
 175.    0.  175.  175.  175.    0.    0.    0.    0.  175.  175.    0.
 174.5   0.  174.5 174.5 174.5 174.5 174.  174.    0.    0.    0.  174.5
 174.    0.    0.  174.5 174.5   0.  174.    0.  173.5 173.  173.    0.
 172.5 173.    0.  173.  173.    0.  173.5   0.    0.  173.5   0.  173.
 173.  173.5 173.  173.    0.  173.  173.    0.    0.    0.    0.    0.
   0.  173.  172.5 172.5 172.5   0.    0.    0.    0.    0.    0.    0.
 173.  172.5 173.  173.    0.  172.5 173.    0.  172.5   0.    0.    0.
   0.    0.    0.    0.    0.  173.  173.  172.5 173.  173.  1

/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume'].replace(0, np.nan, inplace=True)
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:118: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignm

In [7]:
""" 
execute_short_strategy(open_np, close_np, vwap_np, open_time_np, initial_cap, fee_rate, 
                         stop_loss_pct, threshold, entry_time_min, previous_day_last_close) --> equity, trade times
"""

equity = execute_short_strategy(
    np_data['Open'],
    np_data['Close'],
    np_data['VWAP'],
    np_data['open_time'],
    initial_cap=1000000,
    fee_rate=0.001,
    stop_loss_pct=0.04,
    threshold=1.01,
    entry_time_min=546,  # 9:06 AM
    previous_day_last_close=131.5)

equity

/opt/homebrew/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(array([1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1002840, 1000000, 1002840, 1000000,
        1000000, 1000000, 1002840, 1002840, 1000000, 1000000, 1000000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
         999000,  999000,  999000,  999000,  999000,  999000,  999000,
      

In [8]:
calculate_daily_performance(equity[0], trade_count=equity[1]) 

{'Total Trades': 1,
 'Total Return': -0.0010000000000000009,
 'Max Drawdown': 0.0038291252841928802,
 'Win Rate': 0.011152416356877323,
 'Daily Volatility': 0.00042789883983257855}

In [9]:
data.head()

,index,stock_list
0,2022-10-21,[1717]
1,2022-10-28,"[6873, 1904]"
2,2022-11-03,[2520]
3,2022-11-04,[6412]
4,2022-11-09,"[6188, 1409]"


In [10]:
data

,index,stock_list
0,2022-10-21,[1717]
1,2022-10-28,"[6873, 1904]"
2,2022-11-03,[2520]
3,2022-11-04,[6412]
4,2022-11-09,"[6188, 1409]"
...,...,...
154,2024-09-23,[4919]
155,2024-09-24,[2634]
156,2024-09-25,[2606]
157,2024-09-27,[3260]


In [11]:
print(type(data['index']))
print(type(data['stock_list']))
print(type(data['stock_list'][0]))
print(type(data['stock_list'][0][0]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'str'>


In [12]:
import logging

logging.basicConfig(level=logging.DEBUG)


from datetime import datetime

date_to_check = datetime.strptime('2022-10-21', '%Y-%m-%d')
stocks = get_stocks_for_date('target_stocks.feather', date_to_check)
print(f"Stocks for {date_to_check.date()}: {stocks}")


2024-10-23 12:29:21,496 - INFO - [daily_bt.py:258] - get_stocks_for_date - Reading stock list from Feather file: target_stocks.feather
2024-10-23 12:29:21,500 - INFO - [daily_bt.py:305] - get_stocks_for_date - Found 1 stocks for date 2022-10-21


Stocks for 2022-10-21: ['1717']


In [13]:
backtest_single_stock('1717', date_to_check, config)

2024-10-23 12:29:21,504 - INFO - [daily_bt.py:323] - backtest_single_stock - Running backtest for stock: 1717 on date: 2022-10-21
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume'].replace(0, np.nan, inplace=True)
/Users/mouyasushi/Deskto

{'Total Trades': 0,
 'Total Return': 0.0,
 'Max Drawdown': 0.0,
 'Win Rate': 0.0,
 'Daily Volatility': 0.0,
 'Stock': '1717',
 'Date': datetime.date(2022, 10, 21)}

In [14]:
stocks

['1717']

In [15]:
backtest_multiple_stocks(stocks, date_to_check, config)

2024-10-23 12:29:22,194 - INFO - [daily_bt.py:323] - backtest_single_stock - Running backtest for stock: 1717 on date: 2022-10-21
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume'].replace(0, np.nan, inplace=True)
/Users/mouyasushi/Deskto

[{'Total Trades': 0,
  'Total Return': 0.0,
  'Max Drawdown': 0.0,
  'Win Rate': 0.0,
  'Daily Volatility': 0.0,
  'Stock': '1717',
  'Date': datetime.date(2022, 10, 21)}]

In [16]:
main('config.yaml', '2022-10-21', 'target_stocks.feather')

2024-10-23 12:29:22,406 - INFO - [daily_bt.py:258] - get_stocks_for_date - Reading stock list from Feather file: target_stocks.feather
2024-10-23 12:29:22,410 - INFO - [daily_bt.py:305] - get_stocks_for_date - Found 1 stocks for date 2022-10-21
2024-10-23 12:29:22,809 - INFO - [daily_bt.py:323] - backtest_single_stock - Running backtest for stock: 1717 on date: 2022-10-21
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value